# Agent Lab — Quickstart

This notebook demonstrates:
1. Loading free data with yfinance
2. Running Buffett & Momentum agents
3. Combining with OversightAgent


In [ ]:
from datetime import date
import pandas as pd
from agent_lab.data_connectors.market_data import fetch_prices, fetch_fundamentals, build_momentum_features
from agent_lab.agents.buffett import BuffettAgent
from agent_lab.agents.ackman import AckmanAgent
from agent_lab.agents.momentum import MomentumAgent
from agent_lab.agents.congress import CongressAgent
from agent_lab.ensemble.oversight import OversightAgent
from agent_lab.agents.base import Decision

# --- universe and data ---
universe = ["AAPL","MSFT","KO","BAC","JNJ","PG"]
prices = fetch_prices(universe, period="2y")
fund = fetch_fundamentals(universe)
mom = build_momentum_features(prices)

# --- agents ---
buff = BuffettAgent()
ack = AckmanAgent()
mom_agent = MomentumAgent()
cong = CongressAgent()
agents = [buff, ack, mom_agent, cong]

today = date.today()
all_decs = []

# --- gather decisions ---
for agent in agents:
    # select features per agent type
    if isinstance(agent, (BuffettAgent, AckmanAgent)):
        feat = fund
    elif isinstance(agent, MomentumAgent):
        feat = mom
    elif isinstance(agent, CongressAgent):
        feat = pd.DataFrame({
            "recent_congress_buy_usd": [0] * len(universe),
            "recent_congress_sell_usd": [0] * len(universe),
            "days_since_disclosure": [90] * len(universe)
        }, index=universe)
    else:
        raise ValueError(f"Unknown agent type: {type(agent)}")

    # --- call decide and get just the decisions (first element of tuple) ---
    decs, table = agent.decide(today, universe, feat)
    all_decs.extend(decs)

# --- build agent-level CSV ---
rows = []
for d in all_decs:
    row = fund.loc[d.symbol].to_dict()
    rows.append({
        "symbol": d.symbol,
        "Company": row.get("Company"),
        "agent": d.extras.get("agent_name", ""),
        "decision": d.action.name,
        "confidence": d.confidence,
        "score": d.score,
        "justification": d.rationale,
        "sector": d.extras.get("sector", None),
        "asof": d.extras.get("asof", None)
    })

agent_table = pd.DataFrame(rows)
agent_table.set_index(["symbol","agent"], inplace=True)
agent_table.to_csv("all_agents_votes.csv")
print("✅ All agent votes saved to all_agents_votes.csv")
print(agent_table)

# --- prepare dict for OversightAgent ---
all_decs_dict = {s: [d for d in all_decs if d.symbol == s] for s in universe}

# --- combine with OversightAgent ---
over = OversightAgent(agent_weights={
    "BuffettAgent": 1.0,
    "AckmanAgent": 0.9,
    "MomentumAgent": 0.8,
    "CongressAgent": 0.6
})
final_decisions = over.combine(all_decs_dict)

# --- final combined CSV ---
final_rows = []
for sym, d in final_decisions.items():
    final_rows.append({
        "symbol": sym,
        "final_decision": d.action.name,
        "final_confidence": d.confidence,
        "final_score": d.score,
        "rationale": d.rationale
    })

final_table = pd.DataFrame(final_rows)
final_table.set_index("symbol", inplace=True)
final_table.to_csv("final_oversight_decisions.csv")
print("✅ OversightAgent final decisions saved to final_oversight_decisions.csv")
print(final_table)

PermissionError: [Errno 13] Permission denied: 'all_agents_votes.csv'

In [20]:
if __name__ == "__main__":
    # fake fundamentals
    data = pd.DataFrame({
        "pe": [15, 30],
        "roe": [0.2, 0.05],
        "debt_to_equity": [0.5, 2.0],
        "free_cashflow": [100, -50],
        "sector": ["Consumer Defensive", "Tech"]
    }, index=["KO", "AAPL"])

    agent = BuffettAgent()
    decisions, table = agent.decide(date.today(), ["KO","AAPL"], data)

    print("Decisions:")
    for d in decisions:
        print(d)

    print("\nTable:")
    print(table)

Decisions:
Decision(symbol='KO', action=<Action.BUY: 'BUY'>, confidence=0.95, score=8, rationale='P/E<20 | ROE>15% | Low debt | +FreeCF | Sector Consumer Defensive', extras={'sector': 'Consumer Defensive', 'asof': '2025-08-18', 'agent_name': 'BuffettAgent'})
Decision(symbol='AAPL', action=<Action.SELL: 'SELL'>, confidence=0.2, score=0, rationale='P/E high/NA | ROE low | Debt>Eq | No FreeCF | Sector Tech (off)', extras={'sector': 'Tech', 'asof': '2025-08-18', 'agent_name': 'BuffettAgent'})

Table:
        score decision  confidence  \
symbol                               
KO          8      BUY        0.95   
AAPL        0     SELL        0.20   

                                            justification              sector  
symbol                                                                         
KO      P/E<20 | ROE>15% | Low debt | +FreeCF | Sector...  Consumer Defensive  
AAPL    P/E high/NA | ROE low | Debt>Eq | No FreeCF | ...                Tech  
